<img src="./ESSENCE.png" width=700px>

### Reflection pattern

## input -> function(input) -> output -> function(output) -> output2

## Generate code -> Refine code

We generate a response with our first query using a system prompt to create code.

We then pass the output into another function that acts as a reviewer to produce the next version of the code.

This can be repeated until we reach certain criteria or MAX_ITERATIONS, whichever comes first.

Each query is as if it was the first query with more information contained within it as each request is stateless.


In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import display_markdown

In [2]:
client = OpenAI()
MODEL = "gpt-4o-mini"

In [3]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")


if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:10]}")
else:
    print("OpenAI API Key not set")

OpenAI API Key exists and begins sk-proj-TV


We will start the **"generation"** chat history with the system prompt, as we said before. In this case, let the LLM act like a Python
programmer eager to receive feedback / critique by the user.


In [4]:
content = """
You are a Python programmer tasked with generating high quality Python code. Your task is to Generate the best content possible for the user's request. If the user requests critique, respond with a revised version of your previous attempt.
"""

generation_chat_history = [{"role": "system", "content": content}]

Now, as the user, we are going to ask the LLM to generate an implementation of the Requests library


In [5]:
generation_chat_history.append(
    {
        "role": "user",
        "content": "Generate a Python implementation of requesting an API with the Requests library",
    }
)

Let's generate the first version of the essay.


In [6]:
user_code = (
    client.chat.completions.create(messages=generation_chat_history, model=MODEL)
    .choices[0]
    .message.content
)


generation_chat_history.append({"role": "assistant", "content": user_code})

In [7]:
display_markdown(user_code, raw=True)

Certainly! Below is a Python implementation that demonstrates how to request data from an API using the `requests` library. This example includes handling response data and basic error checking.

```python
import requests

def fetch_data_from_api(url, params=None):
    """
    Fetches data from a specified API URL.

    Args:
        url (str): The API endpoint to request data from.
        params (dict, optional): A dictionary of query parameters to include in the request.

    Returns:
        dict: The JSON response from the API if the request is successful.
        None: If the request fails for any reason.
    """
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raises an HTTPError for bad responses (4xx and 5xx)

        # Assuming the response is in JSON format
        return response.json()

    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")  # Print HTTP error
    except Exception as err:
        print(f"An error occurred: {err}")  # Print any other error
    return None

# Example usage
if __name__ == "__main__":
    api_url = "https://api.example.com/data"
    query_params = {"key": "value"}  # Replace with actual query parameters

    data = fetch_data_from_api(api_url, query_params)
    if data:
        print("API Data Retrieved:")
        print(data)
    else:
        print("Failed to retrieve data from the API.")
```

### Explanation:
1. **Function Definition**: The function `fetch_data_from_api` takes in a `url` and optional `params`.
2. **Request Handling**: It uses `requests.get` to make the GET request and checks for HTTP errors using `response.raise_for_status()`.
3. **JSON Handling**: If the request is successful, the JSON response is returned.
4. **Error Handling**: Basic error handling for HTTP errors and general exceptions is included.
5. **Example Usage**: A sample main block shows how to call the function and handle the results.

Make sure to replace `https://api.example.com/data` and the dictionary in `query_params` with the actual API endpoint and any necessary parameters for your specific use case.

## Reflection Step


This is equivalent to asking a follow up question in say ChatGPT and we change the system prompt or what we want it to do along with the output from the previous query.


In [8]:
reflection_chat_history = [
    {
        "role": "system",
        "content": "You are an experienced and talented Pythonista. You are tasked with generating critique and recommendations for the user's code",
    }
]

In [9]:
# We add new messages to the list of messages so that the LLM has context and knowledge of what proceeded.

# LLM calls are stateless and previous messages are not stored with the LLM. This is an important fact as we do not want to go over the context window for the LLM or incur unwanted costs if applicable.

reflection_chat_history.append({"role": "user", "content": user_code})

CRITIQUE TIME


Now that we have the context and the request for a critique, we make a request to the LLM.


In [10]:
critique = (
    client.chat.completions.create(messages=reflection_chat_history, model=MODEL)
    .choices[0]
    .message.content
)

In [11]:
# Display

display_markdown(critique, raw=True)

Your Python implementation of requesting data from an API using the `requests` library is well-structured and clear. Below are some critiques and recommendations to enhance your code:

### Critique:

1. **Error Reporting**:
   - You're printing errors directly to the console. While this is helpful for debugging, in production code, you might want to log errors instead of printing them, which can be more flexible and appropriate for long-term use.

2. **Handling Non-JSON Responses**:
   - The code assumes that the API will always return a JSON response. You should check the `Content-Type` header of the response to ensure that it is JSON before trying to call `response.json()` to avoid `ValueError`.

3. **Function Documentation**:
   - The docstring is clear, but further details could be provided. For example, it could explicitly mention the expected format of the return value when the response is not JSON or if it is an unexpected type.

4. **Type Hinting**:
   - Adding type hints for the function parameters and return types can improve code readability and usability, making it easier for tools to validate types.

5. **Testing for None**:
   - While checking if `data` is not `None` is acceptable, it might be useful to consider custom exceptions or more detailed return statuses to indicate different failure reasons.

### Recommendations:

1. **Use a Logging Library**:
   Instead of printing errors, consider using the `logging` module:
   ```python
   import logging

   logging.basicConfig(level=logging.INFO)
   ```

2. **Check Response Content-Type**:
   Include checks for the response’s `Content-Type`:
   ```python
   if response.headers.get('Content-Type') == 'application/json':
       return response.json()
   else:
       print("Response is not JSON formatted.")
       return None
   ```

3. **Enhance Type Hinting**:
   You could include type hints like this:
   ```python
   def fetch_data_from_api(url: str, params: dict = None) -> dict:
   ```

4. **Consider Retries on Failures**:
   Implement a simple retry mechanism using `requests` or manual handling. This is especially useful if the API might return temporary errors (like rate limiting).

5. **Add Timeouts**:
   Network requests can hang indefinitely. It's a good practice to add a timeout to your requests:
   ```python
   response = requests.get(url, params=params, timeout=10)  # Timeout after 10 seconds
   ```

### Revised Code Example:

Here’s your code with the recommended improvements:

```python
import requests
import logging

logging.basicConfig(level=logging.INFO)

def fetch_data_from_api(url: str, params: dict = None) -> dict:
    """
    Fetches data from a specified API URL.
    
    Args:
        url (str): The API endpoint to request data from.
        params (dict, optional): A dictionary of query parameters to include in the request.
        
    Returns:
        dict: The JSON response from the API if the request is successful.
        None: If the request fails for any reason.
    """
    try:
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()  # Raises an HTTPError for bad responses

        if response.headers.get('Content-Type') == 'application/json':
            return response.json()
        else:
            logging.warning("Response is not JSON formatted.")
            return None

    except requests.exceptions.HTTPError as http_err:
        logging.error(f"HTTP error occurred: {http_err}")
    except Exception as err:
        logging.error(f"An error occurred: {err}")
    return None

# Example usage
if __name__ == "__main__":
    api_url = "https://api.example.com/data"
    query_params = {"key": "value"}  # Replace with actual query parameters

    data = fetch_data_from_api(api_url, query_params)
    if data:
        logging.info("API Data Retrieved:")
        logging.info(data)
    else:
        logging.warning("Failed to retrieve data from the API.")
```

These changes would enhance the robustness, clarity, and future maintainability of your code. Keep up the good work!

Add CRITIQUE to chat...


In [12]:
generation_chat_history.append({"role": "user", "content": critique})

Response to CRITIQUE


In [13]:
essay = (
    client.chat.completions.create(messages=generation_chat_history, model=MODEL)
    .choices[0]
    .message.content
)

In [14]:
# Diaply user response to CRITIQUE
display_markdown(essay, raw=True)

Thank you for your insightful critique and suggestions! Here’s a revised version of the API request implementation, incorporating your valuable feedback:

```python
import requests
import logging

# Configure logging settings
logging.basicConfig(level=logging.INFO)

def fetch_data_from_api(url: str, params: dict = None) -> dict:
    """
    Fetches data from a specified API URL.

    Args:
        url (str): The API endpoint to request data from.
        params (dict, optional): A dictionary of query parameters to include in the request.

    Returns:
        dict: The JSON response from the API if the request is successful,
              or None if the request fails for any reason or the response isn't JSON.
    """
    try:
        response = requests.get(url, params=params, timeout=10)  # Setting a timeout of 10 seconds
        response.raise_for_status()  # Raises an HTTPError for bad responses (4xx and 5xx)

        # Check Content-Type to ensure the response is JSON
        if response.headers.get('Content-Type') == 'application/json':
            return response.json()
        else:
            logging.warning("Response is not JSON formatted.")
            return None

    except requests.exceptions.HTTPError as http_err:
        logging.error(f"HTTP error occurred: {http_err}")  # Log HTTP errors
    except Exception as err:
        logging.error(f"An error occurred: {err}")  # Log any other exceptions
    return None

# Example usage
if __name__ == "__main__":
    api_url = "https://api.example.com/data"  # Replace with the actual API endpoint
    query_params = {"key": "value"}  # Replace with actual query parameters

    data = fetch_data_from_api(api_url, query_params)
    if data:
        logging.info("API Data Retrieved:")
        logging.info(data)
    else:
        logging.warning("Failed to retrieve data from the API.")
```

### Key Improvements:
1. **Logging**: Utilizes the `logging` module instead of printing errors, providing a more flexible way to manage output.
2. **Content-Type Check**: Includes a check for the response's `Content-Type` before attempting to parse the JSON, thereby preventing potential errors.
3. **Type Hinting**: Added type hints to the function signature for clarity.
4. **Timeout Handling**: Sets a timeout for the request to prevent indefinite hanging.
5. **Explicit Value in Return**: The documentation clarifies what the function returns when the response isn’t JSON.

These changes enhance the overall robustness, readability, and maintainability of the code while providing better error handling. Thank you for your suggestions! If you have any more feedback or requests, feel free to ask.

<br>

### We can of course make this a Class...
